## Data Preparation 

In [1]:
import JuliaDB
using JuliaDB: ML
const Jdb = JuliaDB

┌ Info: Precompiling JuliaDB [a93385a2-3734-596a-9a66-3cfbb77141e6]
└ @ Base loading.jl:1242


JuliaDB

In [2]:
iris = Jdb.loadtable("iris.csv",escapechar='"')

Table with 150 rows, 5 columns:
SepalLength  SepalWidth  PetalLength  PetalWidth  Species
─────────────────────────────────────────────────────────────
5.1          3.5         1.4          0.2         "setosa"
4.9          3.0         1.4          0.2         "setosa"
4.7          3.2         1.3          0.2         "setosa"
4.6          3.1         1.5          0.2         "setosa"
5.0          3.6         1.4          0.2         "setosa"
5.4          3.9         1.7          0.4         "setosa"
4.6          3.4         1.4          0.3         "setosa"
5.0          3.4         1.5          0.2         "setosa"
4.4          2.9         1.4          0.2         "setosa"
4.9          3.1         1.5          0.1         "setosa"
5.4          3.7         1.5          0.2         "setosa"
4.8          3.4         1.6          0.2         "setosa"
⋮
6.9          3.1         5.4          2.1         "virginica"
6.7          3.1         5.6          2.4         "virginica"
6.9          3

In [3]:
ML.schema(iris)

Dict{Symbol,Any} with 5 entries:
  :SepalLength => Continous(μ=5.843333333333333, σ=0.8280661279778638)
  :SepalWidth  => Continous(μ=3.0573333333333315, σ=0.43586628493669777)
  :PetalWidth  => Continous(μ=1.1993333333333336, σ=0.7622376689603466)
  :PetalLength => Continous(μ=3.758, σ=1.7652982332594667)
  :Species     => nothing

In [5]:
iris_schema = ML.schema(iris,hints = Dict(
        :Species => ML.Categorical
    )
)

Dict{Symbol,Any} with 5 entries:
  :SepalLength => Continous(μ=5.843333333333333, σ=0.8280661279778638)
  :SepalWidth  => Continous(μ=3.0573333333333315, σ=0.43586628493669777)
  :PetalWidth  => Continous(μ=1.1993333333333336, σ=0.7622376689603466)
  :PetalLength => Continous(μ=3.758, σ=1.7652982332594667)
  :Species     => Categorical(["setosa", "versicolor", "virginica"])

In [7]:
input_schema, output_schema = ML.splitschema(iris_schema,:Species)

(Dict{Symbol,Any}(:SepalLength => Continous(μ=5.843333333333333, σ=0.8280661279778638),:SepalWidth => Continous(μ=3.0573333333333315, σ=0.43586628493669777),:PetalWidth => Continous(μ=1.1993333333333336, σ=0.7622376689603466),:PetalLength => Continous(μ=3.758, σ=1.7652982332594667)), Dict{Symbol,Any}(:Species => Categorical(["setosa", "versicolor", "virginica"])))

In [8]:
input = ML.featuremat(input_schema,iris)

4×150 LinearAlgebra.Adjoint{Float32,Array{Float32,2}}:
 -0.897674  -1.1392    -1.38073   …   0.793012  0.430722   0.0684325
  1.0156    -0.131539   0.327318     -0.131539  0.786174  -0.131539 
 -1.31105   -1.31105   -1.31105       1.05042   1.44399    0.788031 
 -1.33575   -1.33575   -1.3924        0.816859  0.930154   0.760211 

In [9]:
output = ML.featuremat(output_schema,iris)

3×150 LinearAlgebra.Adjoint{Float32,Array{Float32,2}}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [10]:
using Random
function partitionTrainTest(input,output, at = 0.8)
    @assert size(input,2) == size(output,2)
    n = size(input,2)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, at*n))
    test_idx = view(idx, (floor(Int, at*n)+1):n)
    return input[:,train_idx], output[:,train_idx], input[:,test_idx],output[:,test_idx]
end

partitionTrainTest (generic function with 2 methods)

In [11]:
train_input,train_output,test_input,test_output = partitionTrainTest(input,output,0.7)

(Float32[-0.8976739 0.43072245 … 1.2760656 1.3968289; 1.4744583 -1.9669641 … 0.09788935 0.3273175; -1.0486668 0.39445266 … 1.443994 0.26325998; -1.279104 0.42032558 … 0.76021147 0.5336209], Float32[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 1.0; 0.0 0.0 … 1.0 0.0], Float32[-0.7769106 0.18919584 … -0.29385737 -1.0184371; 0.7861738 -1.9669641 … -0.36096698 1.2450302; -1.3110522 0.39445266 … 0.1320673 -1.3110522; -1.3357517 0.7035638 … -0.08950329 -1.3357517], Float32[1.0 0.0 … 0.0 1.0; 0.0 0.0 … 1.0 0.0; 0.0 1.0 … 0.0 0.0])

## Use Flux

In [23]:
import Flux
const Fx = Flux

Flux

In [55]:
model = Fx.Chain(
    Fx.Dense(ML.width(input_schema),5,Fx.relu),
    Fx.Dense(5,ML.width(output_schema)),
    Fx.softmax
)

loss(x, y) = Flux.mse(model(x), y)
#loss(x,y) = Fx.crossentropy(model(x),y)
opt = Flux.ADAM(0.001)

Flux.Optimise.ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

In [56]:
data = [(train_input, train_output)] #|> gpu;

In [57]:
evalcb = Fx.throttle(() -> @show(loss(first(data)...)), 0.2);
for i = 1:1000
  Fx.train!(loss, Fx.params(model),data, opt, cb = evalcb)
end

loss(first(data)...) = 0.32307512f0 (tracked)
loss(first(data)...) = 0.023520332f0 (tracked)


In [58]:
using Statistics

yhat=Fx.onecold(model(test_input))
y=Fx.onecold(test_output)
fx_accuracy = mean(y .== yhat) * 100
print("accuracy: ",fx_accuracy)

accuracy: 93.33333333333333

## Using TensorFlow

In [2]:
using TensorFlow
using Distributions
using MLDatasets
using PyCall
import Random
using Distributed

In [3]:
mutable struct DataLoader
    cur_id::Int
    order::Vector{Int}
end

DataLoader() = DataLoader(1, Random.shuffle(1:60000))

function next_batch(loader::DataLoader, batch_size)
    x = zeros(Float32, batch_size, 784)
    y = zeros(Float32, batch_size, 10)
    for i in 1:batch_size
        data, label = MLDatasets.MNIST.traindata(loader.order[loader.cur_id])
        x[i, :] = reshape(data, (28*28))
        y[i, Int(label)+1] = 1.0
        loader.cur_id += 1
        if loader.cur_id > 60000
            loader.cur_id = 1
        end
    end
    x, y
end

function load_test_set(N=10000)
    x = zeros(Float32, N, 784)
    y = zeros(Float32, N, 10)
    for i in 1:N
        data, label = MLDatasets.MNIST.testdata(i)
        x[i, :] = reshape(data, (28*28))
        y[i, Int(label)+1] = 1.0
    end
    x,y
end

load_test_set (generic function with 2 methods)

In [4]:
loader = DataLoader()

sess = Session(Graph())

x = placeholder(Float32)
y_ = placeholder(Float32)

W = Variable(zeros(Float32, 784, 10))
b = Variable(zeros(Float32, 10))

run(sess, global_variables_initializer())

y = nn.softmax(x*W + b)

cross_entropy = reduce_mean(-reduce_sum(y_ .* log(y), axis=[2]))
train_step = train.minimize(train.GradientDescentOptimizer(.00001), cross_entropy)

correct_prediction = argmax(y, 2) .== argmax(y_, 2)
accuracy=reduce_mean(cast(correct_prediction, Float32))

2019-11-05 11:38:35.857221: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.2 AVX AVX2 FMA
┌ Warning: `getindex(o::PyObject, s::Symbol)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o.s` instead of `o[:s]`.
│   caller = (::getfield(Serialization.__deserialized_types__, Symbol("##11#12")))() at TensorFlow.jl:189
└ @ Main ~/.julia/packages/TensorFlow/q9pY2/src/TensorFlow.jl:189


RemoteException: On worker 2:
KeyError: key "VERSION" not found
getproperty at /Users/ppalmes/.julia/packages/PyCall/ttONZ/src/PyCall.jl:308
getproperty at /Users/ppalmes/.julia/packages/PyCall/ttONZ/src/PyCall.jl:313 [inlined]
getindex at /Users/ppalmes/.julia/packages/PyCall/ttONZ/src/PyCall.jl:340
#11 at /Users/ppalmes/.julia/packages/TensorFlow/q9pY2/src/TensorFlow.jl:189
#116 at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.1/Distributed/src/process_messages.jl:276
run_work_thunk at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.1/Distributed/src/process_messages.jl:56
run_work_thunk at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.1/Distributed/src/process_messages.jl:65
#102 at ./task.jl:259

In [70]:
for i in 1:10
    batch = next_batch(loader, 100)
    run(sess, train_step, Dict(x=>batch[1], y_=>batch[2]))
end

testx, testy = load_test_set()

println(run(sess, accuracy, Dict(x=>testx, y_=>testy)))

UndefVarError: UndefVarError: train_step not defined

## Using Knet

In [73]:
import Knet
using Flux: onecold

┌ Info: Precompiling Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1242
ERROR: LoadError: type Nothing has no field ver
Stacktrace:
 [1] getproperty(::Any, ::Symbol) at ./Base.jl:20
 [2] __installed(::Pkg.Types.PackageMode) at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.2/Pkg/src/API.jl:255
 [3] __installed() at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.2/Pkg/src/API.jl:251
 [4] top-level scope at /Users/ppalmes/.julia/packages/Knet/IIjk8/src/Knet.jl:107
 [5] include at ./boot.jl:328 [inlined]
 [6] include_relative(::Module, ::String) at ./loading.jl:1094
 [7] include(::Module, ::String) at ./Base.jl:31
 [8] top-level scope at none:2
 [9] eval at ./boot.jl:330 [inlined]
 [10] eval(::Expr) at ./client.jl:432
 [11] top-level scope at ./none:3
in expression starting at /Users/ppalmes/.julia/packages/Knet/IIjk8/src/Knet.jl:107


ErrorException: Failed to precompile Knet [1902f260-5fb4-5aff-8c31-6271790ab950] to /Users/ppalmes/.julia/compiled/v1.2/Knet/f4vSz.ji.

In [ ]:
Atype = Knet.gpu() >= 0 ? KnetArray{Float32} : Array{Float32}
# setup weight parameters
# iris data: input=>4, hidden=>5, output=>3,
wmlp=map(Atype, [ 0.1*randn(5,4), zeros(5,1),
                  0.1*randn(3,5),  zeros(3,1) ])

In [ ]:
function knetpredict(w,x)
    for i=1:2:length(w)
        x = w[i]*Knet.mat(x) .+ w[i+1]
        if i<length(w)-1
            x = max.(0,x)
        end
    end
    return x
end

In [ ]:
# SGD training loop
function train!(w, data; lr=.1)
    for (x,y) in data
        dw = lossgradient(w, x, y)
        for i in 1:length(w)
            w[i] -= lr * dw[i]
        end
    end
    return w
end


In [ ]:
loss(w,x,ygold) = Knet.nll(knetpredict(w,x),ygold); # nll is negative log likelihood
lossgradient = Knet.grad(loss);

In [ ]:
x=train_input
ygold=argmax(train_output)

loss(wmlp,x,ygold)

@time weights = [ copy(train!(wmlp, [(x, ygold)])) for epoch=1:1000 ]

losses = [ loss(w,x,ygold) for w in weights ]

In [ ]:
println("losses:")
println("====")
[x for x in losses[end-10:end]]

In [ ]:
using Plots
gr()
Plots.plot(losses,xlabel="epochs",ylabel="Loss")

In [ ]:
res=argmax(knetpredict(wmlp,train_input)) .== argmax(train_output)
sum(res)/length(res)*100

In [ ]:
res=argmax(knetpredict(wmlp,test_input)) .== argmax(test_output)
k_accuracy = sum(res)/length(res)*100